<a href="https://colab.research.google.com/github/Agentoma/Assignment_2_DL_CandyCounter/blob/main/Copy_of_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [ ]:
# Install the required libraries
!pip install transformers datasets evaluate timm albumentations

# Install git-lfs if you plan to push to Hugging Face Hub (optional)
# !sudo apt-get install git-lfs

In [ ]:
import json
import os

def transform_coco_to_metadata(cocodata):
    metadata = []
    for image_info in cocodata['images']:
        # Extract image details
        file_name = image_info['file_name'].split(os.path.sep)[-1]
        image_id = image_info['id']

        # Gather annotations for this image
        annotations = [
            ann for ann in cocodata['annotations'] if ann['image_id'] == image_id
        ]

        # Prepare objects field
        objects = {
            'bbox': [ann['bbox'] for ann in annotations],
            'categories': [ann['category_id'] - 1 for ann in annotations],  # Adjust IDs to start from 0
        }

        # Create metadata entry
        metadata.append({
            'file_name': file_name,
            'objects': objects,
        })
    return metadata


In [ ]:
# Load the COCO annotations
with open('/content/my_data/result.json') as f:
    cocodata = json.load(f)

# Transform to metadata format
metadata = transform_coco_to_metadata(cocodata)

# Write to metadata.jsonl
with open('/content/my_data/metadata.jsonl', 'w') as f:
    for item in metadata:
        json.dump(item, f)
        f.write('\n')


In [ ]:
category_names = [cat['name'] for cat in sorted(cocodata['categories'], key=lambda x: x['id'])]


In [ ]:
from datasets import Features, Sequence, ClassLabel, Value, Image

features = Features({
    'image': Image(),
    'objects': {
        'bbox': Sequence(Sequence(Value(dtype='float32'), length=4)),
        'categories': Sequence(ClassLabel(names=category_names)),
    },
    'file_name': Value('string'),
})


In [ ]:
from datasets import load_dataset

data_dir = '/content/my_data'  # Update if necessary

dataset = load_dataset(
    'imagefolder',
    data_dir=data_dir,
    split='train',
    features=features,
)


Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
def update_image_paths(example):
    example['image'] = os.path.join(data_dir, 'images', example['file_name'])
    return example

dataset = dataset.map(update_image_paths)


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'NoneType'

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)


In [ ]:
from PIL import Image

def load_image(example):
    image_path = example['image']
    example['image'] = Image.open(image_path).convert('RGB')
    return example

dataset['train'] = dataset['train'].map(load_image)
dataset['test'] = dataset['test'].map(load_image)


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

AttributeError: 'Image' object has no attribute 'read'

In [ ]:
print(dataset)
print(dataset['train'][0])


DatasetDict({
    train: Dataset({
        features: ['image', 'objects', 'file_name'],
        num_rows: 16
    })
    test: Dataset({
        features: ['image', 'objects', 'file_name'],
        num_rows: 4
    })
})
{'image': <PIL.Image.Image image mode=RGB size=300x225 at 0x7D357FE46530>, 'objects': {'bbox': [[60.56485366821289, 155.0209197998047, 46.757320404052734, 29.184101104736328], [176.35983276367188, 0.0, 42.050209045410156, 37.34309768676758], [99.16317749023438, 28.556486129760742, 33.5774040222168, 28.87029266357422], [52.71966552734375, 57.74058532714844, 42.364017486572266, 29.497907638549805], [136.5062713623047, 73.11715698242188, 29.497907638549805, 32.949790954589844], [182.32217407226562, 84.72803497314453, 30.753137588500977, 25.41840934753418], [158.47280883789062, 40.167362213134766, 29.811716079711914, 27.615062713623047], [173.849365234375, 111.71548461914062, 28.556486129760742, 29.184101104736328], [151.88284301757812, 156.58995056152344, 26.359832763671875

In [ ]:
import json
import os
from datasets import load_dataset, Dataset, Features, Sequence, ClassLabel, Value

# TEMQUEINSERIR AQUI: Update the path to your zip file if necessary
!unzip -o "/content/contadordedoces.zip" -d /content/my_data  # MUDE AQUI: Ensure your zip file name and path are correct

# The zip file should contain 'images' folder and 'result.json'


Archive:  /content/contadordedoces.zip
  inflating: /content/my_data/result.json  
  inflating: /content/my_data/images/00018539-c9aa2aa3-cd_23.jpg  
  inflating: /content/my_data/images/0404d915-c5dd2c27-cd_34.jpg  
  inflating: /content/my_data/images/0c7bd620-be427c2f-cd_32.jpg  
  inflating: /content/my_data/images/0d73dbdf-cd_35.jpg  
  inflating: /content/my_data/images/1eab85b5-cd_23.jpg  
  inflating: /content/my_data/images/35607d47-cd_22.jpg  
  inflating: /content/my_data/images/39ad0f50-4fba0b24-cd_22.jpg  
  inflating: /content/my_data/images/3a3b2f88-cd_07.jpg  
  inflating: /content/my_data/images/3d2c9b34-cd_04.jpg  
  inflating: /content/my_data/images/4d29b7eb-a3ccc0ee-cd_35.jpg  
  inflating: /content/my_data/images/6c66df32-a1a5846e-cd_04.jpg  
  inflating: /content/my_data/images/87765325-cd_10.jpg  
  inflating: /content/my_data/images/94dad33d-cd_32.jpg  
  inflating: /content/my_data/images/bd6660c8-20c11ac2-cd_07.jpg  
  inflating: /content/my_data/images/ca99f

In [ ]:
# Load the COCO formatted annotations
with open('/content/my_data/result.json') as f:  # MUDE AQUI if your path is different
    cocodata = json.load(f)


In [ ]:
def transform_coco(cocodata):
    """
    Transforms COCO-formatted data to Hugging Face dataset format.

    Args:
        cocodata: A dictionary containing the COCO-formatted data.

    Returns:
        A list of dictionaries, where each dictionary represents an image
        with its annotations in the Hugging Face dataset format.
    """
    huggingdata = []
    for image in cocodata['images']:
        # Remove directory from file name
        image['file_name'] = image['file_name'].split(os.path.sep)[-1]
        image['image_id'] = image['id']

        # Initialize 'objects' dictionary for annotations
        image['objects'] = {'bbox': [], 'category': [], 'area': [], 'id': []}

        # Iterate through annotations and add them to 'objects'
        for annot in cocodata['annotations']:
            if annot['image_id'] == image['id']:
                image['objects']['bbox'].append(annot['bbox'])
                image['objects']['category'].append(annot['category_id'] - 1)  # Adjust category_id to start from 0
                image['objects']['area'].append(annot['area'])
                image['objects']['id'].append(annot['id'])

        # Add the transformed image data to the list
        huggingdata.append(image)

    return huggingdata

# Transform COCO annotations to Hugging Face format
huggingface_data = transform_coco(cocodata)


In [ ]:
# Write the formatted annotations to 'metadata.jsonl'
with open('/content/my_data/metadata.jsonl', 'w') as f:  # MUDE AQUI if your path is different
    for item in huggingface_data:
        f.write(json.dumps(item) + '\n')


In [ ]:
# Create mappings from category IDs to labels and vice versa
id2label = {item['id'] - 1: item['name'] for item in cocodata['categories']}  # Adjust IDs to start from 0
label2id = {v: k for k, v in id2label.items()}


In [ ]:
# Define the features for the dataset
category_names = list(id2label.values())

features = Features({
    'file_name': Value(dtype='string'),
    'image_id': Value(dtype='int64'),
    'width': Value(dtype='int64'),
    'height': Value(dtype='int64'),
    'objects': {
        'bbox': Sequence(feature=Sequence(Value(dtype='float32'), length=4)),  # Each bbox is a list of 4 floats
        'category': Sequence(ClassLabel(names=category_names)),
        'area': Sequence(Value(dtype='float32')),
        'id': Sequence(Value(dtype='int64')),
    }
})


In [ ]:
from datasets import DatasetDict, load_dataset

# Load the dataset from the directory containing 'images' and 'metadata.jsonl'
data_dir = '/content/my_data'  # MUDE AQUI if your path is different

# Load the dataset with specified features
candy_data = load_dataset('imagefolder', data_dir=data_dir, features=features)

# Since the data is not split, we can split it now
candy_data = candy_data['train'].train_test_split(test_size=0.2, seed=42)

# Verify the splits
print(candy_data)


Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file_name', 'image_id', 'width', 'height', 'objects'],
        num_rows: 16
    })
    test: Dataset({
        features: ['file_name', 'image_id', 'width', 'height', 'objects'],
        num_rows: 4
    })
})


In [ ]:
from transformers import AutoImageProcessor

checkpoint = 'facebook/detr-resnet-50'
image_processor = AutoImageProcessor.from_pretrained(checkpoint)


In [ ]:
def formatted_anns(image_id, category, area, bbox):
    annotations = []
    for i in range(len(category)):
        new_ann = {
            'image_id': image_id,
            'category_id': category[i],
            'iscrowd': 0,
            'area': area[i],
            'bbox': list(bbox[i]),
        }
        annotations.append(new_ann)
    return annotations


In [ ]:
import albumentations as A
import numpy as np

# Define the data augmentation transforms
transform = A.Compose(
    [
        A.Resize(480, 480),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
    ],
    bbox_params=A.BboxParams(format='coco', label_fields=['category'])
)


In [ ]:
def transform_aug_ann(examples):
    image_ids = examples['image_id']
    images, bboxes, area, categories = [], [], [], []
    for image, objects in zip(examples['image'], examples['objects']):
        image = np.array(image.convert('RGB'))[:, :, ::-1]  # Convert PIL image to OpenCV format
        out = transform(image=image, bboxes=objects['bbox'], category=objects['category'])
        area.append(objects['area'])
        images.append(out['image'])
        bboxes.append(out['bboxes'])
        categories.append(out['category'])
    targets = [
        {'image_id': id_, 'annotations': formatted_anns(id_, cat_, ar_, box_)}
        for id_, cat_, ar_, box_ in zip(image_ids, categories, area, bboxes)
    ]
    return image_processor(images=images, annotations=targets, return_tensors='pt')


In [ ]:
# Apply the data augmentation and preprocessing to the training set
candy_data['train'] = candy_data['train'].with_transform(transform_aug_ann)


In [ ]:
def transform_ann(examples):
    image_ids = examples['image_id']
    images, bboxes, area, categories = [], [], [], []
    for image, objects in zip(examples['image'], examples['objects']):
        image = np.array(image.convert('RGB'))[:, :, ::-1]  # Convert PIL image to OpenCV format
        # No data augmentation for test data
        out = {'image': image, 'bboxes': objects['bbox'], 'category': objects['category']}
        area.append(objects['area'])
        images.append(out['image'])
        bboxes.append(out['bboxes'])
        categories.append(out['category'])
    targets = [
        {'image_id': id_, 'annotations': formatted_anns(id_, cat_, ar_, box_)}
        for id_, cat_, ar_, box_ in zip(image_ids, categories, area, bboxes)
    ]
    return image_processor(images=images, annotations=targets, return_tensors='pt')


In [ ]:
# Apply the preprocessing to the test set without data augmentation
candy_data['test'] = candy_data['test'].with_transform(transform_ann)


In [ ]:
def collate_fn(batch):
    pixel_values = [item['pixel_values'] for item in batch]
    encoding = image_processor.pad_and_create_pixel_mask(pixel_values, return_tensors='pt')
    labels = [item['labels'] for item in batch]
    batch = {}
    batch['pixel_values'] = encoding['pixel_values']
    batch['pixel_mask'] = encoding['pixel_mask']
    batch['labels'] = labels
    return batch


In [ ]:
from transformers import AutoModelForObjectDetection

model = AutoModelForObjectDetection.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='candy_detector',
    per_device_train_batch_size=2,
    num_train_epochs=125,  # Adjust the number of epochs as needed
    fp16=True,
    save_steps=200,
    logging_steps=10,
    learning_rate=1e-5,
    weight_decay=1e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
)


In [ ]:
# @title
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=candy_data['train'],
    eval_dataset=candy_data['test'],
    tokenizer=image_processor,
)

trainer.train()


KeyError: 'image'

In [ ]:
# Save the fine-tuned model
trainer.save_model('candy_detector')


In [ ]:
def candy_counter(image):
    from transformers import pipeline
    # TEMQUEINSERIR AQUI: Update the path to your saved model if necessary
    model_path = 'candy_detector'  # MUDE AQUI if your model path is different
    obj_detector = pipeline('object-detection', model=model_path, threshold=0.1)  # Adjust threshold if needed
    # Ensure the image is a Numpy array
    if not isinstance(image, np.ndarray):
        image = np.array(image)
    results = obj_detector(image)
    counts = {label: 0 for label in id2label.values()}
    for res in results:
        label = res['label']
        counts[label] += 1
    return counts


In [ ]:
def collate_fn(batch):
    images = [item['image'] for item in batch]
    pixel_values = [image_processor(image, return_tensors="pt")["pixel_values"][0] for image in images]

    encoding = image_processor.pad_and_create_pixel_mask(pixel_values, return_tensors='pt')
    labels = [item['labels'] for item in batch]
    batch = {}
    batch['pixel_values'] = encoding['pixel_values']
    batch['pixel_mask'] = encoding['pixel_mask']
    batch['labels'] = labels

    return batch

In [ ]:
print(candy_data['train'].column_names)
print(candy_data['test'].column_names)

['file_name', 'image_id', 'width', 'height', 'objects']
['file_name', 'image_id', 'width', 'height', 'objects']


In [ ]:
from PIL import Image

def load_image(example):
    image_path = example['file_name']  # Get the image file path
    example['image'] = Image.open(image_path).convert('RGB')  # Load the image using PIL
    return example

# Apply the load_image function to both train and test datasets
candy_data['train'] = candy_data['train'].map(load_image)
candy_data['test'] = candy_data['test'].map(load_image)

# Ensure collate_fn can handle the 'image' data
# ... (previous suggestion for collate_fn remains the same)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

KeyError: 'image'

In [ ]:
from PIL import Image

def load_image(example):
    example['image'] = Image.open(example['filepath'])
    return example

candy_data['train'] = candy_data['train'].map(load_image)
candy_data['test'] = candy_data['test'].map(load_image)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

KeyError: 'image'

In [ ]:
from PIL import Image
import numpy as np

# TEMQUEINSERIR AQUI: Update the path to your test image
test_image_path = '/content/my_data/images/your_test_image.jpg'  # MUDE AQUI: Provide your test image path
image = Image.open(test_image_path)
image_np = np.array(image)

counts = candy_counter(image_np)
print(counts)


FileNotFoundError: [Errno 2] No such file or directory: '/content/my_data/images/your_test_image.jpg'

In [ ]:
from transformers import pipeline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# TEMQUEINSERIR AQUI: Update the path to your saved model if necessary
model_path = 'candy_detector'  # MUDE AQUI if your model path is different
obj_detector = pipeline('object-detection', model=model_path, threshold=0.1)  # Adjust threshold if needed

results = obj_detector(image_np)

fig, ax = plt.subplots(1, figsize=(12, 8))
ax.imshow(image_np)

for res in results:
    box = res['box']
    x = box['xmin']
    y = box['ymin']
    width = box['xmax'] - x
    height = box['ymax'] - y
    rect = patches.Rectangle(
        (x, y), width, height, linewidth=2, edgecolor='red', facecolor='none'
    )
    ax.add_patch(rect)
    plt.text(
        x,
        y,
        res['label'],
        color='white',
        fontsize=12,
        bbox=dict(facecolor='red', alpha=0.5),
    )

plt.axis('off')
plt.show()


In [ ]:
# Optional: Evaluate the model using COCO metrics
# This requires additional setup and is optional for the assignment
# You can refer to the Hugging Face tutorial for guidance


# Object detection

Object detection is the computer vision task of detecting instances (such as humans, buildings, or cars) in an image. Object detection models receive an image as input and output
coordinates of the bounding boxes and associated labels of the detected objects. An image can contain multiple objects,
each with its own bounding box and a label (e.g. it can have a car and a building), and each object can
be present in different parts of an image (e.g. the image can have several cars).
This task is commonly used in autonomous driving for detecting things like pedestrians, road signs, and traffic lights.
Other applications include counting objects in images, image search, and more.

In this guide, you will learn how to:

 1. Finetune [DETR](https://huggingface.co/docs/transformers/model_doc/detr), a model that combines a convolutional
 backbone with an encoder-decoder Transformer, on the [candycounter]
 dataset.
 2. Use your finetuned model for inference.

<Tip>
The task illustrated in this tutorial is supported by the following model architectures:

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[Conditional DETR](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/conditional_detr), [Deformable DETR](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/deformable_detr), [DETA](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/deta), [DETR](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/detr), [Table Transformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/table-transformer), [YOLOS](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/yolos)

<!--End of the generated tip-->

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install -q datasets transformers evaluate timm albumentations
```

You'll use 🤗 Datasets to load a dataset from the Hugging Face Hub, 🤗 Transformers to train your model,
and `albumentations` to augment the data. `timm` is currently required to load a convolutional backbone for the DETR model.

We encourage you to share your model with the community. Log in to your Hugging Face account to upload it to the Hub.
When prompted, enter your token to log in:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load the Candycounter dataset

The Contadorededoces contains images with
annotations identifying candy in the context of the candyassignment.

Start by loading the dataset:

In [ ]:
# Install the required libraries
!pip install transformers datasets evaluate timm albumentations

# Install git-lfs if you plan to push to Hugging Face Hub (optional)
# !sudo apt-get install git-lfs


In [ ]:
import json
import os
from datasets import load_dataset, Features, Sequence, ClassLabel, Value

# Unzip the dataset (if not already unzipped)
!unzip -o "/content/contadordedoces.zip" -d /content/my_data

# Load the COCO formatted annotations
with open('/content/my_data/result.json') as f:
    cocodata = json.load(f)

# Load the dataset using imagefolder
candy_data = load_dataset('imagefolder', data_dir="/content/my_data/images")


In [ ]:
import json
import os

def transform_coco(cocodata):
  """Transforms COCO-formatted data to Hugging Face dataset format.

  Args:
    cocodata: A dictionary containing the COCO-formatted data.

  Returns:
    A list of dictionaries, where each dictionary represents an image
    with its annotations in the Hugging Face dataset format.
  """
  huggingdata = []
  for image in cocodata['images']:
    # 1. Update file_name and image_id for compatibility
    image['file_name'] = image['file_name'].split(os.path.sep)[-1]  # Remove directory from file name
    image['image_id'] = image['id']  # Rename 'id' to 'image_id'

    # 2. Initialize 'objects' dictionary for annotations
    image['objects'] = {'bbox': [], 'category': [], 'area': [], 'id': []}

    # 3. Iterate through annotations and add them to 'objects'
    for annot in cocodata['annotations']:
        if annot['image_id'] == image['id']:
            image['objects']['bbox'].append(annot['bbox'])
            image['objects']['category'].append(annot['category_id'])
            image['objects']['area'].append(annot['area'])
            image['objects']['id'].append(annot['id'])

    # 4. Add the transformed image data to the list
    huggingdata.append(image)

  return huggingdata

# Load the COCO formatted annotations
with open('/content/my_data/result.json') as f:
    cocodata = json.load(f)

# Transform to Hugging Face format
huggingface_data = transform_coco(cocodata)

# Now you can load this data into a Hugging Face dataset
# For example:
# from datasets import Dataset
# dataset = Dataset.from_list(huggingface_data)

In [ ]:
import json
import os
from datasets import Dataset, Features, Sequence, ClassLabel, Value

# ... (your transform_coco function from the previous response) ...

# Load the COCO formatted annotations
with open('/content/my_data/result.json') as f:
    cocodata = json.load(f)

# Transform to Hugging Face format
huggingface_data = transform_coco(cocodata)

# Define features for the dataset
features = Features({
    'file_name': Value(dtype='string'),
    'image_id': Value(dtype='int64'),
    'width': Value(dtype='int64'),
    'height': Value(dtype='int64'),
    'objects': {
        'bbox': Sequence(feature=Sequence(feature=Value(dtype='float32'))),  # Bounding boxes (COCO format)
        'category': Sequence(feature=ClassLabel(names=['Moon', 'Insect', 'Black_star', 'Grey_star', 'Unicorn_whole', 'Unicorn_head', 'Owl', 'Cat'])),  # Replace with your category names
        'area': Sequence(feature=Value(dtype='float32')),
        'id': Sequence(feature=Value(dtype='int64')),
    }
})

# Load the data into a Hugging Face Dataset
dataset = Dataset.from_list(huggingface_data, features=features)

# Now you can work with the dataset:
print(dataset)
print(dataset[0])  # Access the first example
# ...

In [ ]:
# Load the COCO formatted annotations
with open('/content/my_data/result.json') as f:  # MUDE AQUI if your path is different
    cocodata = json.load(f)


In [ ]:
# Print the first few elements of the 'images' list:
print(cocodata['images'][:5])
# Or print a specific image:
print(cocodata['images'][0])

# Explore other parts of the data similarly

In [ ]:
# Convert COCO annotations to a format readable by Hugging Face Datasets
huggingdata = []
for image in cocodata['images']:
    # Remove the image directory from the file name
    image['file_name'] = image['file_name'].split(os.path.sep)[-1]
    image['image_id'] = image['id']
    # Extend the image dict with bounding boxes and class labels
    image['objects'] = {'bbox': [], 'category': [], 'area': [], 'id': []}
    # Iterate through the annotations (bounding boxes and labels)
    for annot in cocodata['annotations']:
        if annot['image_id'] == image['id']:
            # Add the annotation
            image['objects']['bbox'].append(annot['bbox'])
            image['objects']['category'].append(annot['category_id'])
            image['objects']['area'].append(annot['area'])
            image['objects']['id'].append(annot['id'])
    # Append the image dict with annotations to the list
    huggingdata.append(image)


In [ ]:
import json
import os
from datasets import load_dataset, Features, Value, Sequence, ClassLabel

!unzip -o contadordedoces.zip -d /content/my_data  # Extract to /content/my_data folder

# ... and then update the file paths in your code accordingly
with open('/content/my_data/result.json') as f:
    cocodata = json.load(f)

candy_data = load_dataset('imagefolder', data_dir="/content/my_data/images")


# Load the COCO JSON data (with the correct path)
with open('/content/my_data/result.json') as f:
    cocodata = json.load(f)

# Category mapping (from your assignment)
id2label = {
    1: 'Moon',
    2: 'Insect',
    3: 'Black_star',
    4: 'Grey_star',
    5: 'Unicorn_whole',
    6: 'Unicorn_head',
    7: 'Owl',
    8: 'Cat'
}

# ... (your transform_coco function remains the same)

# Define features
features = Features({
    # ... (your feature definitions)
})

# Load the dataset
candy_data = load_dataset('imagefolder', data_dir="images", features=features)


You'll see that this dataset already comes with a training set containing 1000 images and a test set with 29 images.

To get familiar with the data, explore what the examples look like.

In [ ]:
cppe5["train"][0]

The examples in the dataset have the following fields:
- `image_id`: the example image id
- `image`: a `PIL.Image.Image` object containing the image
- `width`: width of the image
- `height`: height of the image
- `objects`: a dictionary containing bounding box metadata for the objects in the image:
  - `id`: the annotation id
  - `area`: the area of the bounding box
  - `bbox`: the object's bounding box (in the [COCO format](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/#coco) )
  - `category`: the object's category, with possible values including `Coverall (0)`, `Face_Shield (1)`, `Gloves (2)`, `Goggles (3)` and `Mask (4)`

You may notice that the `bbox` field follows the COCO format, which is the format that the DETR model expects.
However, the grouping of the fields inside `objects` differs from the annotation format DETR requires. You will
need to apply some preprocessing transformations before using this data for training.

To get an even better understanding of the data, visualize an example in the dataset.

In [ ]:
import numpy as np
import os
from PIL import Image, ImageDraw

image = cppe5["train"][0]["image"]
annotations = cppe5["train"][0]["objects"]
draw = ImageDraw.Draw(image)

categories = cppe5["train"].features["objects"].feature["category"].names

id2label = {index: x for index, x in enumerate(categories, start=0)}
label2id = {v: k for k, v in id2label.items()}

for i in range(len(annotations["id"])):
    box = annotations["bbox"][i - 1]
    class_idx = annotations["category"][i - 1]
    x, y, w, h = tuple(box)
    draw.rectangle((x, y, x + w, y + h), outline="red", width=1)
    draw.text((x, y), id2label[class_idx], fill="white")

image

<div class="flex justify-center">
    <img src="https://i.imgur.com/TdaqPJO.png" alt="CPPE-5 Image Example"/>
</div>

To visualize the bounding boxes with associated labels, you can get the labels from the dataset's metadata, specifically
the `category` field.
You'll also want to create dictionaries that map a label id to a label class (`id2label`) and the other way around (`label2id`).
You can use them later when setting up the model. Including these maps will make your model reusable by others if you share
it on the Hugging Face Hub.

As a final step of getting familiar with the data, explore it for potential issues. One common problem with datasets for
object detection is bounding boxes that "stretch" beyond the edge of the image. Such "runaway" bounding boxes can raise
errors during training and should be addressed at this stage. There are a few examples with this issue in this dataset.
To keep things simple in this guide, we remove these images from the data.

In [ ]:
remove_idx = [590, 821, 822, 875, 876, 878, 879]
keep = [i for i in range(len(cppe5["train"])) if i not in remove_idx]
cppe5["train"] = cppe5["train"].select(keep)

## Preprocess the data

To finetune a model, you must preprocess the data you plan to use to match precisely the approach used for the pre-trained model.
[AutoImageProcessor](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoImageProcessor) takes care of processing image data to create `pixel_values`, `pixel_mask`, and
`labels` that a DETR model can train with. The image processor has some attributes that you won't have to worry about:

- `image_mean = [0.485, 0.456, 0.406 ]`
- `image_std = [0.229, 0.224, 0.225]`

These are the mean and standard deviation used to normalize images during the model pre-training. These values are crucial
to replicate when doing inference or finetuning a pre-trained image model.

Instantiate the image processor from the same checkpoint as the model you want to finetune.

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "facebook/detr-resnet-50"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

Before passing the images to the `image_processor`, apply two preprocessing transformations to the dataset:
- Augmenting images
- Reformatting annotations to meet DETR expectations

First, to make sure the model does not overfit on the training data, you can apply image augmentation with any data augmentation library. Here we use [Albumentations](https://albumentations.ai/docs/) ...
This library ensures that transformations affect the image and update the bounding boxes accordingly.
The 🤗 Datasets library documentation has a detailed [guide on how to augment images for object detection](https://huggingface.co/docs/datasets/object_detection),
and it uses the exact same dataset as an example. Apply the same approach here, resize each image to (480, 480),
flip it horizontally, and brighten it:

In [ ]:
import albumentations
import numpy as np
import torch

transform = albumentations.Compose(
    [
        albumentations.Resize(480, 480),
        albumentations.HorizontalFlip(p=1.0),
        albumentations.RandomBrightnessContrast(p=1.0),
    ],
    bbox_params=albumentations.BboxParams(format="coco", label_fields=["category"]),
)

The `image_processor` expects the annotations to be in the following format: `{'image_id': int, 'annotations': List[Dict]}`,
 where each dictionary is a COCO object annotation. Let's add a function to reformat annotations for a single example:

In [ ]:
def formatted_anns(image_id, category, area, bbox):
    annotations = []
    for i in range(0, len(category)):
        new_ann = {
            "image_id": image_id,
            "category_id": category[i],
            "isCrowd": 0,
            "area": area[i],
            "bbox": list(bbox[i]),
        }
        annotations.append(new_ann)

    return annotations

Now you can combine the image and annotation transformations to use on a batch of examples:

In [ ]:
# transforming a batch
def transform_aug_ann(examples):
    image_ids = examples["image_id"]
    images, bboxes, area, categories = [], [], [], []
    for image, objects in zip(examples["image"], examples["objects"]):
        image = np.array(image.convert("RGB"))[:, :, ::-1]
        out = transform(image=image, bboxes=objects["bbox"], category=objects["category"])

        area.append(objects["area"])
        images.append(out["image"])
        bboxes.append(out["bboxes"])
        categories.append(out["category"])

    targets = [
        {"image_id": id_, "annotations": formatted_anns(id_, cat_, ar_, box_)}
        for id_, cat_, ar_, box_ in zip(image_ids, categories, area, bboxes)
    ]

    return image_processor(images=images, annotations=targets, return_tensors="pt")

Apply this preprocessing function to the entire dataset using 🤗 Datasets [with_transform](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.with_transform) method. This method applies
transformations on the fly when you load an element of the dataset.

At this point, you can check what an example from the dataset looks like after the transformations. You should see a tensor
with `pixel_values`, a tensor with `pixel_mask`, and `labels`.

In [ ]:
cppe5["train"] = cppe5["train"].with_transform(transform_aug_ann)
cppe5["train"][15]

You have successfully augmented the individual images and prepared their annotations. However, preprocessing isn't
complete yet. In the final step, create a custom `collate_fn` to batch images together.
Pad images (which are now `pixel_values`) to the largest image in a batch, and create a corresponding `pixel_mask`
to indicate which pixels are real (1) and which are padding (0).

In [ ]:
def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    encoding = image_processor.pad_and_create_pixel_mask(pixel_values, return_tensors="pt")
    labels = [item["labels"] for item in batch]
    batch = {}
    batch["pixel_values"] = encoding["pixel_values"]
    batch["pixel_mask"] = encoding["pixel_mask"]
    batch["labels"] = labels
    return batch

## Training the DETR model

You have done most of the heavy lifting in the previous sections, so now you are ready to train your model!
The images in this dataset are still quite large, even after resizing. This means that finetuning this model will
require at least one GPU.

Training involves the following steps:
1. Load the model with [AutoModelForObjectDetection](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForObjectDetection) using the same checkpoint as in the preprocessing.
2. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments).
3. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, image processor, and data collator.
4. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

When loading the model from the same checkpoint that you used for the preprocessing, remember to pass the `label2id`
and `id2label` maps that you created earlier from the dataset's metadata. Additionally, we specify `ignore_mismatched_sizes=True` to replace the existing classification head with a new one.

In [ ]:
from transformers import AutoModelForObjectDetection

model = AutoModelForObjectDetection.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

In the [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) use `output_dir` to specify where to save your model, then configure hyperparameters as you see fit.
It is important you do not remove unused columns because this will drop the image column. Without the image column, you
can't create `pixel_values`. For this reason, set `remove_unused_columns` to `False`.
If you wish to share your model by pushing to the Hub, set `push_to_hub` to `True` (you must be signed in to Hugging
Face to upload your model).

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="detr-resnet-50_finetuned_cppe5",
    per_device_train_batch_size=8,
    num_train_epochs=10,
    fp16=True,
    save_steps=200,
    logging_steps=50,
    learning_rate=1e-5,
    weight_decay=1e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=True,
)

Finally, bring everything together, and call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=cppe5["train"],
    tokenizer=image_processor,
)

trainer.train()

If you have set `push_to_hub` to `True` in the `training_args`, the training checkpoints are pushed to the
Hugging Face Hub. Upon training completion, push the final model to the Hub as well by calling the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method.

In [ ]:
trainer.push_to_hub()

## Evaluate

Object detection models are commonly evaluated with a set of <a href="https://cocodataset.org/#detection-eval">COCO-style metrics</a>.
You can use one of the existing metrics implementations, but here you'll use the one from `torchvision` to evaluate the final
model that you pushed to the Hub.

To use the `torchvision` evaluator, you'll need to prepare a ground truth COCO dataset. The API to build a COCO dataset
requires the data to be stored in a certain format, so you'll need to save images and annotations to disk first. Just like
when you prepared your data for training, the annotations from the `cppe5["test"]` need to be formatted. However, images
should stay as they are.

The evaluation step requires a bit of work, but it can be split in three major steps.
First, prepare the `cppe5["test"]` set: format the annotations and save the data to disk.

In [ ]:
import json


# format annotations the same as for training, no need for data augmentation
def val_formatted_anns(image_id, objects):
    annotations = []
    for i in range(0, len(objects["id"])):
        new_ann = {
            "id": objects["id"][i],
            "category_id": objects["category"][i],
            "iscrowd": 0,
            "image_id": image_id,
            "area": objects["area"][i],
            "bbox": objects["bbox"][i],
        }
        annotations.append(new_ann)

    return annotations


# Save images and annotations into the files torchvision.datasets.CocoDetection expects
def save_cppe5_annotation_file_images(cppe5):
    output_json = {}
    path_output_cppe5 = f"{os.getcwd()}/cppe5/"

    if not os.path.exists(path_output_cppe5):
        os.makedirs(path_output_cppe5)

    path_anno = os.path.join(path_output_cppe5, "cppe5_ann.json")
    categories_json = [{"supercategory": "none", "id": id, "name": id2label[id]} for id in id2label]
    output_json["images"] = []
    output_json["annotations"] = []
    for example in cppe5:
        ann = val_formatted_anns(example["image_id"], example["objects"])
        output_json["images"].append(
            {
                "id": example["image_id"],
                "width": example["image"].width,
                "height": example["image"].height,
                "file_name": f"{example['image_id']}.png",
            }
        )
        output_json["annotations"].extend(ann)
    output_json["categories"] = categories_json

    with open(path_anno, "w") as file:
        json.dump(output_json, file, ensure_ascii=False, indent=4)

    for im, img_id in zip(cppe5["image"], cppe5["image_id"]):
        path_img = os.path.join(path_output_cppe5, f"{img_id}.png")
        im.save(path_img)

    return path_output_cppe5, path_anno

Next, prepare an instance of a `CocoDetection` class that can be used with `cocoevaluator`.

In [ ]:
import torchvision


class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, feature_extractor, ann_file):
        super().__init__(img_folder, ann_file)
        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        # read in PIL image and target in COCO format
        img, target = super(CocoDetection, self).__getitem__(idx)

        # preprocess image and target: converting target to DETR format,
        # resizing + normalization of both image and target)
        image_id = self.ids[idx]
        target = {"image_id": image_id, "annotations": target}
        encoding = self.feature_extractor(images=img, annotations=target, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze()  # remove batch dimension
        target = encoding["labels"][0]  # remove batch dimension

        return {"pixel_values": pixel_values, "labels": target}


im_processor = AutoImageProcessor.from_pretrained("MariaK/detr-resnet-50_finetuned_cppe5")

path_output_cppe5, path_anno = save_cppe5_annotation_file_images(cppe5["test"])
test_ds_coco_format = CocoDetection(path_output_cppe5, im_processor, path_anno)

Finally, load the metrics and run the evaluation.

In [ ]:
import evaluate
from tqdm import tqdm

model = AutoModelForObjectDetection.from_pretrained("MariaK/detr-resnet-50_finetuned_cppe5")
module = evaluate.load("ybelkada/cocoevaluate", coco=test_ds_coco_format.coco)
val_dataloader = torch.utils.data.DataLoader(
    test_ds_coco_format, batch_size=8, shuffle=False, num_workers=4, collate_fn=collate_fn
)

with torch.no_grad():
    for idx, batch in enumerate(tqdm(val_dataloader)):
        pixel_values = batch["pixel_values"]
        pixel_mask = batch["pixel_mask"]

        labels = [
            {k: v for k, v in t.items()} for t in batch["labels"]
        ]  # these are in DETR format, resized + normalized

        # forward pass
        outputs = model(pixel_values=pixel_values, pixel_mask=pixel_mask)

        orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
        results = im_processor.post_process(outputs, orig_target_sizes)  # convert outputs of model to COCO api

        module.add(prediction=results, reference=labels)
        del batch

results = module.compute()
print(results)

These results can be further improved by adjusting the hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). Give it a go!

## Inference

Now that you have finetuned a DETR model, evaluated it, and uploaded it to the Hugging Face Hub, you can use it for inference.
The simplest way to try out your finetuned model for inference is to use it in a [Pipeline](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.Pipeline). Instantiate a pipeline
for object detection with your model, and pass an image to it:

In [ ]:
from transformers import pipeline
import requests

url = "https://i.imgur.com/2lnWoly.jpg"
image = Image.open(requests.get(url, stream=True).raw)

obj_detector = pipeline("object-detection", model="MariaK/detr-resnet-50_finetuned_cppe5")
obj_detector(image)

You can also manually replicate the results of the pipeline if you'd like:

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("MariaK/detr-resnet-50_finetuned_cppe5")
model = AutoModelForObjectDetection.from_pretrained("MariaK/detr-resnet-50_finetuned_cppe5")

with torch.no_grad():
    inputs = image_processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    target_sizes = torch.tensor([image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.5, target_sizes=target_sizes)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

Let's plot the result:

In [ ]:
draw = ImageDraw.Draw(image)

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    x, y, x2, y2 = tuple(box)
    draw.rectangle((x, y, x2, y2), outline="red", width=1)
    draw.text((x, y), model.config.id2label[label.item()], fill="white")

image

<div class="flex justify-center">
    <img src="https://i.imgur.com/4QZnf9A.png" alt="Object detection result on a new image"/>
</div>